In [1]:
import numpy as np
import matplotlib.pyplot as plt
import time

from CSRandom import CSRandomLite as CSRandom
from ObjectInfo import ObjectInfo
from MushroomFloor import findMushroomFloor
from TrashCans import checkTrash, GarbageLocations
from MonsterFloor import findMonsterFloors
from geode_crack import getTreasureFromGeode, getNumTreasureFromGeode
from Utility import seasonDict, dayToYSD
from Crops import harvest_quality, find_best_days

In [2]:
import pickle
import os

try:
    monster_mushroom_seeds
    with open('data/monster_mushroom_seeds.pkl','wb') as f:
        pickle.dump(monster_mushroom_seeds, f)
except:
    if os.path.exists('data/monster_mushroom_seeds.pkl'):
        with open('data/monster_mushroom_seeds.pkl','rb') as f:
            monster_mushroom_seeds = pickle.load(f)

# Legacy
```
new_seeds = [
    214281694, 214281695, 214283076, 214283077, 214286644, 214286645, 214288026, 214288027,
    214292976, 214292977, 214294358, 214294359, 214299308, 214299309]
gameID = 214286645
```

# Testing new timing

In [ ]:
dayToGo = 5
seed = 1458
monster_mushroom_seeds = [seed for seed in monster_seeds if 81 in findMushroomFloor(seed, dayToGo)]
print(monster_mushroom_seeds)

In [5]:
# seed = 3866
print(findMonsterFloors(351326, 9), findMushroomFloor(351326, 9))

[(13, 'Monster Area'), (57, 'Slime Area'), (103, 'Monster Area')] [81, 82]


In [ ]:
dayToGo = 5

nseeds = 10**4
monster_seeds = [seed for seed in range(nseeds) if len(findMonsterFloors(seed, dayToGo)) == 0]
print(monster_seeds)

In [ ]:
dayToGo = 9
floorsToSpawn = {81,82}

if False:
    nseeds_list = 10**np.arange(5)

    for nseeds in nseeds_list:
        t1 = time.clock()
        mushroom_seeds = [seed for seed in range(nseeds) if len(floorsToSpawn.difference(findMushroomFloor(seed, dayToGo))) == 0]
        t2 = time.clock()
        print(nseeds, ':', t2-t1, 'seconds', '(', nseeds/(t2-t1), ')')

# 1 : 0.0002249999999999197 seconds ( 4444.44444445 )
# 10 : 0.002047000000000132 seconds ( 4885.19785051 )
# 100 : 0.019970000000000265 seconds ( 5007.5112669 )
# 1000 : 0.18659399999999993 seconds ( 5359.22912848 )
# 10000 : 1.8011289999999995 seconds ( 5552.07317188 )

In [7]:
dayToGo = 9
floorsToSpawn = {81,82}
if True:
    nseeds = 10**7
    t1 = time.perf_counter()
    mushroom_seeds = [seed for seed in range(nseeds) if len(floorsToSpawn.difference(findMushroomFloor(seed, dayToGo))) == 0]
    monster_mushroom_seeds = [seed for seed in mushroom_seeds if not any([floor <= min(floorsToSpawn) for (floor, floor_type) in findMonsterFloors(seed, dayToGo)])]
    t2 = time.perf_counter()
    print(nseeds, ':', t2-t1, 'seconds', '(', nseeds/(t2-t1), ')', 'down to', len(monster_mushroom_seeds))

# 10000000 : 1793.2794789999998 seconds ( 5576.375638657493 ) down to 7020

10000000 : 1741.453080735 seconds ( 5742.330993941788 ) down to 320


In [9]:
monster_mushroom_seeds, len(monster_mushroom_seeds)

([3280,
  3281,
  6044,
  6045,
  20090,
  20091,
  34136,
  34137,
  218082,
  218083,
  232128,
  232129,
  246174,
  246175,
  248938,
  248939,
  446930,
  446931,
  460976,
  460977,
  475022,
  475023,
  673014,
  673015,
  687060,
  687061,
  689824,
  689825,
  823068,
  823069,
  901862,
  901863,
  915908,
  915909,
  918672,
  918673,
  1127946,
  1127947,
  1130710,
  1130711,
  1144756,
  1144757,
  1278000,
  1278001,
  1356794,
  1356795,
  1359558,
  1359559,
  1370840,
  1370841,
  1504084,
  1504085,
  1506848,
  1506849,
  1585642,
  1585643,
  1718886,
  1718887,
  1732932,
  1732933,
  1735696,
  1735697,
  1811726,
  1811727,
  1947734,
  1947735,
  1961780,
  1961781,
  2173818,
  2173819,
  2176582,
  2176583,
  2187864,
  2187865,
  2190628,
  2190629,
  2388620,
  2388621,
  2402666,
  2402667,
  2416712,
  2416713,
  2419476,
  2419477,
  2614704,
  2614705,
  2617468,
  2617469,
  2628750,
  2628751,
  2631514,
  2631515,
  2645560,
  2645561,
  2843552,
  2

# GEODE TRACKING

In [10]:
geode_seeds = []
for seed in monster_mushroom_seeds:
#     treasure = [getNumTreasureFromGeode(seed, i+1, 3) for i in range(1)]
#     if not any(['Fire Quartz' in t[0] for t in treasure]):
#         continue
#     if not any(['Gold Ore' in t[0] for t in treasure]):
#         continue    
#     if not any(['Copper Ore' in t[0] for t in treasure]):
#         continue    
    geode_seeds.append(seed)
#     print(seed, [(t[0].split('/')[0], t[1]) for t in treasure])

In [11]:
len(geode_seeds)

320

# CROP TRACKING

In [12]:
mailbox = 68,16
x_range = np.arange(58, 70)
n_x = len(x_range)
y_range = np.arange(18, 28)
n_y = len(y_range)
farmer_level = 0
fertilizer = None
season = 1

for seed in geode_seeds:
    farm_land = 'Seed: {}\n'.format(seed)
    block = ' '.join(['HHH' for x in x_range if x < mailbox[0]-5]+['SSS SSS SSS HHH HHH MBX']+['PPP' for x in x_range if x > mailbox[0]])
    block += '\n' + ' '.join(['PPP' for _ in x_range])
    farm_land += block
    for y_idx in range(n_y):
        y = y_range[y_idx]
        row_vals = []
        for x_idx in range(n_x):
            x = x_range[x_idx]
            quality, days = find_best_days(seed, season, farmer_level, x, y, fertilizer)
            if quality != 2:
                row_vals.append('___')
            else:
                row_vals.append('G{0:02d}'.format(days[0]))
        farm_land += '\n' + ' '.join(row_vals)
    if ('G09' in farm_land) and ('G10' in farm_land) and ('G13' in farm_land):
        print(farm_land+'\n')

Seed: 3280
HHH HHH HHH HHH HHH SSS SSS SSS HHH HHH MBX PPP
PPP PPP PPP PPP PPP PPP PPP PPP PPP PPP PPP PPP
G08 ___ ___ ___ ___ ___ ___ G26 G19 G12 G05 ___
___ ___ ___ ___ ___ ___ G22 G15 G08 ___ ___ ___
___ ___ ___ ___ G25 G18 G11 ___ ___ ___ ___ ___
___ ___ G28 G21 G14 G07 ___ ___ ___ ___ ___ ___
___ G24 G17 G10 ___ ___ ___ ___ ___ ___ ___ ___
G20 G13 G06 ___ ___ ___ ___ ___ ___ ___ ___ ___
G09 ___ ___ ___ ___ ___ ___ ___ ___ ___ ___ ___
___ ___ ___ ___ ___ ___ ___ ___ ___ ___ ___ ___
___ ___ ___ ___ ___ ___ ___ ___ ___ ___ ___ ___
___ ___ ___ ___ ___ ___ ___ ___ ___ ___ ___ ___

Seed: 3281
HHH HHH HHH HHH HHH SSS SSS SSS HHH HHH MBX PPP
PPP PPP PPP PPP PPP PPP PPP PPP PPP PPP PPP PPP
G07 ___ ___ ___ ___ ___ ___ G25 G18 G11 ___ ___
___ ___ ___ ___ ___ G28 G21 G14 G07 ___ ___ ___
___ ___ ___ ___ G24 G17 G10 ___ ___ ___ ___ ___
___ ___ G27 G20 G13 G06 ___ ___ ___ ___ ___ ___
___ G23 G16 G09 ___ ___ ___ ___ ___ ___ ___ ___
G19 G12 G05 ___ ___ ___ ___ ___ ___ ___ ___ ___
G08 ___ ___ ___ _

Seed: 687061
HHH HHH HHH HHH HHH SSS SSS SSS HHH HHH MBX PPP
PPP PPP PPP PPP PPP PPP PPP PPP PPP PPP PPP PPP
___ G23 G16 G09 ___ ___ ___ ___ ___ ___ G27 G20
G19 G12 G05 ___ ___ ___ ___ ___ ___ G23 G16 G09
G08 ___ ___ ___ ___ ___ ___ G26 G19 G12 G05 ___
___ ___ ___ ___ ___ ___ G22 G15 G08 ___ ___ ___
___ ___ ___ ___ G25 G18 G11 ___ ___ ___ ___ ___
___ ___ G28 G21 G14 G07 ___ ___ ___ ___ ___ ___
___ G24 G17 G10 ___ ___ ___ ___ ___ ___ G28 G21
G20 G13 G06 ___ ___ ___ ___ ___ ___ G24 G17 G10
G09 ___ ___ ___ ___ ___ ___ G27 G20 G13 G06 ___
___ ___ ___ ___ ___ ___ G23 G16 G09 ___ ___ ___

Seed: 689824
HHH HHH HHH HHH HHH SSS SSS SSS HHH HHH MBX PPP
PPP PPP PPP PPP PPP PPP PPP PPP PPP PPP PPP PPP
___ ___ ___ ___ G26 G19 G12 G05 ___ ___ ___ ___
___ ___ ___ G22 G15 G08 ___ ___ ___ ___ ___ ___
___ G25 G18 G11 ___ ___ ___ ___ ___ ___ ___ G22
G21 G14 G07 ___ ___ ___ ___ ___ ___ G25 G18 G11
G10 ___ ___ ___ ___ ___ ___ G28 G21 G14 G07 ___
___ ___ ___ ___ ___ ___ G24 G17 G10 ___ ___ ___
___ ___ ___ _

Seed: 1370841
HHH HHH HHH HHH HHH SSS SSS SSS HHH HHH MBX PPP
PPP PPP PPP PPP PPP PPP PPP PPP PPP PPP PPP PPP
___ ___ ___ ___ ___ ___ ___ ___ ___ ___ ___ ___
___ ___ ___ ___ ___ ___ ___ ___ ___ ___ ___ ___
___ ___ ___ ___ ___ ___ ___ ___ ___ ___ G28 G21
___ ___ ___ ___ ___ ___ ___ ___ ___ G24 G17 G10
___ ___ ___ ___ ___ ___ ___ G27 G20 G13 G06 ___
___ ___ ___ ___ ___ ___ G23 G16 G09 ___ ___ ___
___ ___ ___ ___ G26 G19 G12 G05 ___ ___ ___ ___
___ ___ ___ G22 G15 G08 ___ ___ ___ ___ ___ ___
___ G25 G18 G11 ___ ___ ___ ___ ___ ___ ___ G22
G21 G14 G07 ___ ___ ___ ___ ___ ___ G25 G18 G11

Seed: 1504084
HHH HHH HHH HHH HHH SSS SSS SSS HHH HHH MBX PPP
PPP PPP PPP PPP PPP PPP PPP PPP PPP PPP PPP PPP
G08 ___ ___ ___ ___ ___ ___ G26 G19 G12 G05 ___
___ ___ ___ ___ ___ ___ G22 G15 G08 ___ ___ ___
___ ___ ___ ___ G25 G18 G11 ___ ___ ___ ___ ___
___ ___ G28 G21 G14 G07 ___ ___ ___ ___ ___ ___
___ G24 G17 G10 ___ ___ ___ ___ ___ ___ ___ ___
G20 G13 G06 ___ ___ ___ ___ ___ ___ ___ ___ ___
G09 ___ ___

Seed: 2187865
HHH HHH HHH HHH HHH SSS SSS SSS HHH HHH MBX PPP
PPP PPP PPP PPP PPP PPP PPP PPP PPP PPP PPP PPP
___ G23 G16 G09 ___ ___ ___ ___ ___ ___ G27 G20
G19 G12 G05 ___ ___ ___ ___ ___ ___ G23 G16 G09
G08 ___ ___ ___ ___ ___ ___ G26 G19 G12 G05 ___
___ ___ ___ ___ ___ ___ G22 G15 G08 ___ ___ ___
___ ___ ___ ___ G25 G18 G11 ___ ___ ___ ___ ___
___ ___ G28 G21 G14 G07 ___ ___ ___ ___ ___ ___
___ G24 G17 G10 ___ ___ ___ ___ ___ ___ G28 G21
G20 G13 G06 ___ ___ ___ ___ ___ ___ G24 G17 G10
G09 ___ ___ ___ ___ ___ ___ G27 G20 G13 G06 ___
___ ___ ___ ___ ___ ___ G23 G16 G09 ___ ___ ___

Seed: 2190628
HHH HHH HHH HHH HHH SSS SSS SSS HHH HHH MBX PPP
PPP PPP PPP PPP PPP PPP PPP PPP PPP PPP PPP PPP
___ ___ ___ ___ G26 G19 G12 G05 ___ ___ ___ ___
___ ___ ___ G22 G15 G08 ___ ___ ___ ___ ___ ___
___ G25 G18 G11 ___ ___ ___ ___ ___ ___ ___ G22
G21 G14 G07 ___ ___ ___ ___ ___ ___ G25 G18 G11
G10 ___ ___ ___ ___ ___ ___ G28 G21 G14 G07 ___
___ ___ ___ ___ ___ ___ G24 G17 G10 ___ ___ ___
___ ___ ___

Seed: 3069637
HHH HHH HHH HHH HHH SSS SSS SSS HHH HHH MBX PPP
PPP PPP PPP PPP PPP PPP PPP PPP PPP PPP PPP PPP
G07 ___ ___ ___ ___ ___ ___ G25 G18 G11 ___ ___
___ ___ ___ ___ ___ G28 G21 G14 G07 ___ ___ ___
___ ___ ___ ___ G24 G17 G10 ___ ___ ___ ___ ___
___ ___ G27 G20 G13 G06 ___ ___ ___ ___ ___ ___
___ G23 G16 G09 ___ ___ ___ ___ ___ ___ G27 G20
G19 G12 G05 ___ ___ ___ ___ ___ ___ G23 G16 G09
G08 ___ ___ ___ ___ ___ ___ G26 G19 G12 G05 ___
___ ___ ___ ___ ___ ___ G22 G15 G08 ___ ___ ___
___ ___ ___ ___ G25 G18 G11 ___ ___ ___ ___ ___
___ ___ G28 G21 G14 G07 ___ ___ ___ ___ ___ ___

Seed: 3072400
HHH HHH HHH HHH HHH SSS SSS SSS HHH HHH MBX PPP
PPP PPP PPP PPP PPP PPP PPP PPP PPP PPP PPP PPP
___ G24 G17 G10 ___ ___ ___ ___ ___ ___ G28 G21
G20 G13 G06 ___ ___ ___ ___ ___ ___ G24 G17 G10
G09 ___ ___ ___ ___ ___ ___ G27 G20 G13 G06 ___
___ ___ ___ ___ ___ ___ G23 G16 G09 ___ ___ ___
___ ___ ___ ___ G26 G19 G12 G05 ___ ___ ___ ___
___ ___ ___ G22 G15 G08 ___ ___ ___ ___ ___ ___
___ G25 G18

...

Seed: 6740978
HHH HHH HHH HHH HHH SSS SSS SSS HHH HHH MBX PPP
PPP PPP PPP PPP PPP PPP PPP PPP PPP PPP PPP PPP
G08 ___ ___ ___ ___ ___ ___ G26 G19 G12 G05 ___
___ ___ ___ ___ ___ ___ G22 G15 G08 ___ ___ ___
___ ___ ___ ___ G25 G18 G11 ___ ___ ___ ___ ___
___ ___ G28 G21 G14 G07 ___ ___ ___ ___ ___ ___
___ G24 G17 G10 ___ ___ ___ ___ ___ ___ G28 G21
G20 G13 G06 ___ ___ ___ ___ ___ ___ G24 G17 G10
G09 ___ ___ ___ ___ ___ ___ G27 G20 G13 G06 ___
___ ___ ___ ___ ___ ___ G23 G16 G09 ___ ___ ___
___ ___ ___ ___ G26 G19 G12 G05 ___ ___ ___ ___
___ ___ ___ G22 G15 G08 ___ ___ ___ ___ ___ ___

Seed: 6740979
HHH HHH HHH HHH HHH SSS SSS SSS HHH HHH MBX PPP
PPP PPP PPP PPP PPP PPP PPP PPP PPP PPP PPP PPP
G07 ___ ___ ___ ___ ___ ___ G25 G18 G11 ___ ___
___ ___ ___ ___ ___ G28 G21 G14 G07 ___ ___ ___
___ ___ ___ ___ G24 G17 G10 ___ ___ ___ ___ ___
___ ___ G27 G20 G13 G06 ___ ___ ___ ___ ___ ___
___ G23 G16 G09 ___ ___ ___ ___ ___ ___ G27 G20
G19 G12 G05 ___ ___ ___ ___ ___ ___ G23 G16 G09
G08 ___ ___

Seed: 7181865
HHH HHH HHH HHH HHH SSS SSS SSS HHH HHH MBX PPP
PPP PPP PPP PPP PPP PPP PPP PPP PPP PPP PPP PPP
G07 ___ ___ ___ ___ ___ ___ G25 G18 G11 ___ ___
___ ___ ___ ___ ___ G28 G21 G14 G07 ___ ___ ___
___ ___ ___ ___ G24 G17 G10 ___ ___ ___ ___ ___
___ ___ G27 G20 G13 G06 ___ ___ ___ ___ ___ ___
___ G23 G16 G09 ___ ___ ___ ___ ___ ___ ___ ___
G19 G12 G05 ___ ___ ___ ___ ___ ___ ___ ___ ___
G08 ___ ___ ___ ___ ___ ___ ___ ___ ___ ___ ___
___ ___ ___ ___ ___ ___ ___ ___ ___ ___ ___ ___
___ ___ ___ ___ ___ ___ ___ ___ ___ ___ ___ ___
___ ___ ___ ___ ___ ___ ___ ___ ___ ___ ___ ___

Seed: 7184628
HHH HHH HHH HHH HHH SSS SSS SSS HHH HHH MBX PPP
PPP PPP PPP PPP PPP PPP PPP PPP PPP PPP PPP PPP
___ G24 G17 G10 ___ ___ ___ ___ ___ ___ G28 G21
G20 G13 G06 ___ ___ ___ ___ ___ ___ G24 G17 G10
G09 ___ ___ ___ ___ ___ ___ G27 G20 G13 G06 ___
___ ___ ___ ___ ___ ___ G23 G16 G09 ___ ___ ___
___ ___ ___ ___ G26 G19 G12 G05 ___ ___ ___ ___
___ ___ ___ G22 G15 G08 ___ ___ ___ ___ ___ ___
___ G25 G18

Seed: 8080446
HHH HHH HHH HHH HHH SSS SSS SSS HHH HHH MBX PPP
PPP PPP PPP PPP PPP PPP PPP PPP PPP PPP PPP PPP
___ G24 G17 G10 ___ ___ ___ ___ ___ ___ G28 G21
G20 G13 G06 ___ ___ ___ ___ ___ ___ G24 G17 G10
G09 ___ ___ ___ ___ ___ ___ G27 G20 G13 G06 ___
___ ___ ___ ___ ___ ___ G23 G16 G09 ___ ___ ___
___ ___ ___ ___ G26 G19 G12 G05 ___ ___ ___ ___
___ ___ ___ G22 G15 G08 ___ ___ ___ ___ ___ ___
___ G25 G18 G11 ___ ___ ___ ___ ___ ___ ___ ___
G21 G14 G07 ___ ___ ___ ___ ___ ___ ___ ___ ___
G10 ___ ___ ___ ___ ___ ___ ___ ___ ___ ___ ___
___ ___ ___ ___ ___ ___ ___ ___ ___ ___ ___ ___

Seed: 8080447
HHH HHH HHH HHH HHH SSS SSS SSS HHH HHH MBX PPP
PPP PPP PPP PPP PPP PPP PPP PPP PPP PPP PPP PPP
___ G23 G16 G09 ___ ___ ___ ___ ___ ___ G27 G20
G19 G12 G05 ___ ___ ___ ___ ___ ___ G23 G16 G09
G08 ___ ___ ___ ___ ___ ___ G26 G19 G12 G05 ___
___ ___ ___ ___ ___ ___ G22 G15 G08 ___ ___ ___
___ ___ ___ ___ G25 G18 G11 ___ ___ ___ ___ ___
___ ___ G28 G21 G14 G07 ___ ___ ___ ___ ___ ___
___ G24 G17

Seed: 9126319
HHH HHH HHH HHH HHH SSS SSS SSS HHH HHH MBX PPP
PPP PPP PPP PPP PPP PPP PPP PPP PPP PPP PPP PPP
___ ___ ___ ___ ___ ___ ___ G25 G18 G11 ___ ___
___ ___ ___ ___ ___ G28 G21 G14 G07 ___ ___ ___
___ ___ ___ ___ G24 G17 G10 ___ ___ ___ ___ ___
___ ___ G27 G20 G13 G06 ___ ___ ___ ___ ___ ___
___ G23 G16 G09 ___ ___ ___ ___ ___ ___ G27 G20
G19 G12 G05 ___ ___ ___ ___ ___ ___ G23 G16 G09
G08 ___ ___ ___ ___ ___ ___ G26 G19 G12 G05 ___
___ ___ ___ ___ ___ ___ G22 G15 G08 ___ ___ ___
___ ___ ___ ___ G25 G18 G11 ___ ___ ___ ___ ___
___ ___ G28 G21 G14 G07 ___ ___ ___ ___ ___ ___

Seed: 9140364
HHH HHH HHH HHH HHH SSS SSS SSS HHH HHH MBX PPP
PPP PPP PPP PPP PPP PPP PPP PPP PPP PPP PPP PPP
___ G24 G17 G10 ___ ___ ___ ___ ___ ___ G28 G21
G20 G13 G06 ___ ___ ___ ___ ___ ___ G24 G17 G10
G09 ___ ___ ___ ___ ___ ___ G27 G20 G13 G06 ___
___ ___ ___ ___ ___ ___ G23 G16 G09 ___ ___ ___
___ ___ ___ ___ G26 G19 G12 G05 ___ ___ ___ ___
___ ___ ___ G22 G15 G08 ___ ___ ___ ___ ___ ___
___ G25 G18

Seed: 9581250
HHH HHH HHH HHH HHH SSS SSS SSS HHH HHH MBX PPP
PPP PPP PPP PPP PPP PPP PPP PPP PPP PPP PPP PPP
___ G24 G17 G10 ___ ___ ___ ___ ___ ___ ___ ___
G20 G13 G06 ___ ___ ___ ___ ___ ___ ___ ___ ___
G09 ___ ___ ___ ___ ___ ___ ___ ___ ___ ___ ___
___ ___ ___ ___ ___ ___ ___ ___ ___ ___ ___ ___
___ ___ ___ ___ ___ ___ ___ ___ ___ ___ ___ ___
___ ___ ___ ___ ___ ___ ___ ___ ___ ___ ___ ___
___ ___ ___ ___ ___ ___ ___ ___ ___ ___ ___ ___
___ ___ ___ ___ ___ ___ ___ ___ ___ ___ ___ ___
___ ___ ___ ___ ___ ___ ___ ___ ___ ___ ___ ___
___ ___ ___ ___ ___ ___ ___ ___ ___ ___ ___ ___

Seed: 9595297
HHH HHH HHH HHH HHH SSS SSS SSS HHH HHH MBX PPP
PPP PPP PPP PPP PPP PPP PPP PPP PPP PPP PPP PPP
___ ___ ___ ___ G25 G18 G11 ___ ___ ___ ___ ___
___ ___ G28 G21 G14 G07 ___ ___ ___ ___ ___ ___
___ G24 G17 G10 ___ ___ ___ ___ ___ ___ ___ ___
G20 G13 G06 ___ ___ ___ ___ ___ ___ ___ ___ ___
G09 ___ ___ ___ ___ ___ ___ ___ ___ ___ ___ ___
___ ___ ___ ___ ___ ___ ___ ___ ___ ___ ___ ___
___ ___ ___

#  TRASH TRACKING

In [ ]:
all_items = set()
for seed in geode_seeds:
    trash_list = []
    for day, furnace in [(5,False), (9, False), (10, True), (17,True)]:
        day_items = list(filter(lambda x: x[0], [(checkTrash(seed, day, index, item[0][0], item[0][1], furnace),item[1]) for index,item in GarbageLocations.items()]))
        if day_items:
            trash_list.append((day,day_items))
            all_items = all_items.union([d[0] for d in day_items])
    if not any(['Catfish' in t for trash in trash_list for t in trash[1]]):
        continue
    print(seed, trash_list)
print(all_items)

# RNG Tracking

In [ ]:
intercepts = np.array([0.7262432699679598,0.8173253595909687,0.7680226893946633,0.5581611914365372,0.2060331540210327,0.5588847946184151,0.9060270660119257,0.4421778733107158,0.9775497531413798,0.2737044576898703,0.2919062847699533,0.4673147003479836,0.6326590728166788,0.4695118784296848,0.9821512531406019,0.0303669907294060,0.8623701538249712,0.9953470812157481,0.6771811492169189,0.3145917930242568,0.8169079086822029,0.8480517830923440,0.9919021753556571,0.0326251983794501,0.6999419837724148,0.5262841426424143,0.9340186589090240,0.6876202824933548,0.5468154342597422,0.0811099498910410,0.1871245741784222,0.4533271852197718,0.2971718657282981,0.9885437791182398,0.6426974728902325,0.7629635882391425,0.0303942929163548,0.3810045068995116,0.3431418446559189,0.9574551656644117,0.5051292127487851,0.7159725123625120,0.1189577291342233,0.2734514834701323,0.9070979500688138,0.7947655938541357,0.3371603392703273,0.4572087877696420,0.1468250402933103,0.2213147395389690,0.4100732795941053,0.7187268327543171,0.6198302836249724,0.4879680524989813,0.1949134367494441,0.8781914868756158,0.8254231842353117,0.7353974910152132,0.8582192076641224,0.6797490113786184,0.6248661357093911,0.2184067835185708,0.8953624390509736,0.8964398032503389,0.0865798835114482,0.8385790995501816,0.1701428346196854,0.6441152936984390,0.8268144055394523,0.2191876649014595,0.9999726978130511,0.4813656469254595,0.6522052365598294,0.7197259835525072,0.8094625802754716,0.1009353963196908,0.7290940539581208,0.7184506918855247,0.1255272483106364,0.9051763899182791,0.1891238033720868,0.4768098711393820,0.5407952422000446,0.3255006947207733,0.6710366702969357,0.4683977414241050,0.8334969015947994,0.8092038132293168,0.7936303423687956,0.7645059860146167,0.9375404040038308,0.2949968019011416,0.5227852992353892,0.6633928332773004,0.3325890299550207,0.2867224292302144,0.8206100733115385,0.2225179258838845,0.1868715999586841,0.0685188505186320,0.6246227592344502,0.6930450455718884,0.6303943822301898,0.9276373753918509,0.2213420417259177,0.9287076326686459,0.0665215961944878,0.9001043000724652,0.6785054722235098,0.0939780404297532,0.1490974329174950,0.1069724923497870,0.6098702427045770,0.9530428177458433,0.4906252080065316,0.1480562645700090,0.6776115413185263,0.5698617443302003,0.2254031329534032,0.6181821420873432,0.0050821979553821,0.3609390213903687,0.8504849513296433,0.0623084195248356,0.2816472608976286,0.7049758959119097,0.9585803476900702,0.9888124032825290,0.3871369535974865,0.5227401510452574,0.2803253230081523,0.5065761280742362,0.5315790919268406,0.0570083977920042,0.2805536306838290,0.4960787578001985,0.8464154889091922,0.6131578668081936,0.1041586529948557,0.7423290376282898,0.4018761452296171,0.9333926015223342,0.1306983410058069,0.6996523019390424,0.6154085530971216,0.8305679116540439,0.6851265591965646,0.5697424814895459,0.1727676252707688,0.1845327653850115,0.6091108879116880,0.2507483289813381,0.9971147929304814,0.5686894578713408,0.0634366525632499,0.2636837378440815,0.8425600942422450,0.5680859627053543,0.6459901144942222,0.5163661458140082,0.9701272849785757,0.0777091929119589,0.5129673464749787,0.1557653211782526,0.8136527174216009,0.6425213048432588,0.5753934004229463,0.5528618449125726,0.6724891870620144,0.9945464502063330,0.3016407756608169,0.0644536745103326,0.4657030913353446,0.4830740953251133,0.2163059968577261,0.0716895964330480,0.2302406803845618,0.1508326493906009,0.4468998664277140,0.4510793492435847,0.0198493367153449,0.3888378662005243,0.8160447780117601,0.2026041882124749,0.9136292631335693,0.0295769940268140,0.5094613351437549,0.9628896340554998,0.9935717452287542,0.0168698928397475,0.6535186635579535,0.2783295262038379,0.9671677523139715,0.5877925071808474,0.7722017526497141,0.3241669523176582,0.4204252550473555,0.9749330198275544,0.8859995845174415,0.9728872482538630,0.2551745112310977,0.1322647142839919,0.8972532944275314,0.1782211750644358,0.8828919897241946,0.5446572134013554,0.7850452376459935,0.5140406976053680,0.3523834610136150,0.9917470561302021,0.0334430574595197,0.6917274448516442,0.1211860962776403,0.1949107652506377,0.4965168090986631,0.5812894187780513,0.2616644149001988,0.3103631582625039,0.2421360580446832,0.7840757233947868,0.1330599696995038,0.6125037663674465,0.5592900996838185,0.6556192942222669,0.3410277866483795,0.0233112494569789,0.0972859221963612,0.8779105841544972,0.7108723426753992,0.8921390445400676,0.6512643413856926,0.2553076605570073,0.2648330648731594,0.4740126181738510,0.1959048380124870,0.8875846224313531,0.4915845717729929,0.6378236029473243,0.3197121984882803,0.3689720497322138,0.2445317563808205,0.9954206375383868,0.6105061730418848,0.0018246890985523,0.9834268353802277,0.9617912187063095,0.1571434299261978,0.7722569870633338,0.0912756980821843,0.1909123338716627,0.0625025374174595,0.1100620967848516,0.7327969617828713,0.1019238611226547,0.8398272785543590,0.6426707448636511,0.5729746146001735,0.2416340002052644,0.8371933884160563,0.8595807402672155,0.4473712912049943,0.9071346534914965,0.8031683549299689,0.4602444164735472,0.3404827147445095,0.7781353969025124,0.4268943799784846,0.6471734781037892,0.9990059272381505,0.6089321866673102,0.0897048470050585,0.6238408119528744,0.9906509597742236,0.8731640083124693,0.5395439670139663,0.1376393321611170,0.0019975933255616,0.5574654105852663,0.6721924588420394,0.3792365679420701,0.8661678195307813,0.3250289351330273,0.7866348860723129,0.5199600088037364,0.8296365071226082,0.5412022446008409,0.5225106987741360,0.1629092037505047,0.6341853396194919,0.5532340931488360,0.3146100078311795,0.2499505715677284,0.8006302145312681,0.4601314582210646,0.9216007585272197,0.3373971028893241,0.1922522826084179,0.1502617565683377,0.6613419743540426,0.2052914384777151,0.5348968387278248,0.5099699518224085,0.7732510598251834,0.4823435114148741,0.1012074868666042,0.4386617254645851,0.1194111370106280,0.8596329534704019,0.5623798941086884,0.7021879463932421,0.6406731515380895,0.0155092040149072,0.5694415413632251,0.4579568204739861,0.9934129207364344,0.1223423560719670,0.1204997674191834,0.2832083461262325,0.6306079093509390,0.7992804701436685,0.2556246981283765,0.2639851762279799,0.0129881384842974,0.4457718340893144,0.2943221788361305,0.8397542754373301,0.8232105974216064,0.5305195015531590,0.9515632497852498,0.2021468641246422,0.9453870495526991,0.8517358367572240,0.8961234362312236,0.4669010203643240,0.8443397292142454,0.3561978677083728,0.5517778753078439,0.3042913746574387,0.4187525219371322,0.3909747816580231,0.4217911075902130,0.6628777453037341,0.6005293096418163,0.9319973932262497,0.9125609416107465,0.2991008038162724,0.6805090302045034,0.3426733940572819,0.4667185374846303,0.7992584024552527,0.2168973354701407,0.9090439364821855,0.5196538472173986,0.8620615042103741,0.9496667403493387,0.2709116624998449,0.4969818133381110,0.3274792257358690,0.1880213325787434,0.2614532114292743,0.6154511280429787,0.5888916354574690,0.9080697036851522,0.3602330299840462,0.7568008968405430,0.7889373147808655,0.1193857677836836,0.1025405209989010,0.6136170912597408,0.6372150530280616,0.5705644807641229,0.8162083927617447,0.8644558241891004,0.2396331276929160,0.3774893625534557,0.5927469528246424,0.6634558665861636,0.0809907452580476,0.5332108924785680,0.9952213750198583,0.1592452452328266,0.4805372033643245,0.0638009640685286,0.1523048473299968,0.9852495286545015,0.0363431130705136,0.3320819895398253,0.0340619320208495,0.5172342800149854,0.5734280667144005,0.8592160543702617,0.2242986495719750,0.1162700420786953,0.1572993105078579,0.7755236536150443,0.8328994721327441,0.7548080416185818,0.2402962796577701,0.1751964963857069,0.1236236268298812,0.1797150360325888,0.5779685092056024,0.7290563027975412,0.8295034844565687,0.2286939216911299,0.4006889427083958,0.0445881122930851,0.2800207195244826,0.4845721416569184,0.3569197875246963,0.6074557959136813,0.4159910680800636,0.7942683332573010,0.1927999575588852,0.1022079661964477,0.1349139246786544,0.5250906713889403,0.7557648563551553,0.3749835013295447,0.7204577837700292,0.4568553252410401,0.0853238357628341,0.5853062409839156,0.0401890245453402,0.7779989986577998,0.3462658311921478,0.6999383506830494,0.7071565136812425,0.4641094740778717,0.5445898904207116,0.8379389112060606,0.4231595869283935,0.9057942488723407,0.4095872656486870,0.8155063389872697,0.5812767360272243,0.7514809005667832,0.2342974796119600,0.9236109256388670,0.5845605859461056,0.9917550007774285,0.0520612700153427,0.5494897903639310,0.1603144924902891,0.9659722708007191,0.4432249862901982,0.4300303163146741,0.9234700845198101,0.0550913443114103,0.6406097289363899,0.3078088007438038,0.9990431852634266,0.8653127783282254,0.4547387126156775,0.6667683015888409,0.0943912002697547,0.9926622682216867,0.6888672782522007,0.0515044857987690,0.8824280904989821,0.7007505920253464,0.3374315986118426,0.8159112454466109,0.9399822512362069,0.5189808763186359,0.1842962089852878,0.5101968192077228,0.3846810676086141,0.3772936185716156,0.5209312301692233,0.3834330241118711,0.2907931917769804,0.8321539307162883,0.7904229153834390,0.7287027829926008,0.4047940552256974,0.5358340453989031,0.4249917484936265,0.0742167537446212,0.3347740123676015,0.9162355148774737,0.7764682661632394,0.6520651693698323,0.8234997451414818,0.2367810896769078,0.8388957259426340,0.5578468086001681,0.4510555362566633,0.7428189640598460,0.7211151387175150,0.5886144678055376,0.0626136223145823,0.1827929938131911,0.0411828351398851,0.8838099939207593,0.6543234021655859,0.2361500245687319,0.6095075391277240,0.6413336161716532,0.7816760618154315])
offsets = np.array([0.5224253141891330,0.2934186175900599,0.6989879904775822,0.2134429305854453,0.4514857397654494,0.8738978066825762,0.4480566975884405,0.5016844028149191,0.1237167004140637,0.3687510976422350,0.7367305279414776,0.7807144451796609,0.6874510588531618,0.5202552692593332,0.6999718657228967,0.6245208962934657,0.4203590701428983,0.6199993270542468,0.0270703863478593,0.3872048824034655,0.1328705382220776,0.2453366975511130,0.1690050103557320,0.3493458793262700,0.0980057134749395,0.6431830104641537,0.8597649637888021,0.6196079596968406,0.2762129960005232,0.8053595907079797,0.3691777416361391,0.2634068221149066,0.4021239664415476,0.0250706235063591,0.3491786235706781,0.0433380166270481,0.8283493061681974,0.6814988179511849,0.1712519089557472,0.5678030785954572,0.7676959455794171,0.2778238376033603,0.5721466986332773,0.1131640007315036,0.5272359775971790,0.8806399362537266,0.3402213344072092,0.0156658948472077,0.0976937092364270,0.1321693710666939,0.1161793666501434,0.4507004541581034,0.7267575285987729,0.2037269120121965,0.2002296630294200,0.2770886166380200,0.1244136072343278,0.3496421111513124,0.1154372171105059,0.8083027293012954,0.0141328428937740,0.8284487378916001,0.2254714068143961,0.3183571097060839,0.9995733560060956,0.4733237058265712,0.3785904787381135,0.6623804353468028,0.1710766456886551,0.6566338490958483,0.7961715901252679,0.7388602521917133,0.4487474180984998,0.4592673077524023,0.6195089368240483,0.8550467006187171,0.6731899989178356,0.0558410096242284,0.8221099017290909,0.2173657772212130,0.3029616760569446,0.8440990689415944,0.5219142504604133,0.1440436249338294,0.6891802240578366,0.9184772874780358,0.5366492935161337,0.1983970544293508,0.8248409604769388,0.0720900069326582,0.9189244093927201,0.4787071950168848,0.5660616008406790,0.3629491796544517,0.5536702357016831,0.9392472076878170,0.0523524307889642,0.2537895889271934,0.1135906447254078,0.0539122717706078,0.5020494575156127,0.6778408990604061,0.8445892491585527,0.4410598601405786,0.3359977809414256,0.3773191144584302,0.0019530360596036,0.2674902208463709,0.5842179751881480,0.3451829624107027,0.6038986177201843,0.0685725976100995,0.5275322094222215,0.8980714398892928,0.5053410532443509,0.1700337739521795,0.3065344874311865,0.0814277818805667,0.6291768856482474,0.0810960685280599,0.9366744123104377,0.1801934243087626,0.8375394748698637,0.0989866387559970,0.7377094397031277,0.3174643951083834,0.1727986513510340,0.0857982384440481,0.9055970720507194,0.6802617291362313,0.8026942698297531,0.4194004099906425,0.9422503648988205,0.7681976299584833,0.7153163197055999,0.6251207769965383,0.9995098197830424,0.0808543903198346,0.8080458439924035,0.3118611095994064,0.9165242514184325,0.2691590726697627,0.6141790792412029,0.4796579980662363,0.4681913892124738,0.8503518117826206,0.9511749855946634,0.6679901609513864,0.8576081264101008,0.3836364617495036,0.6327127202566307,0.9709246489083972,0.6246127032789462,0.0324945761973479,0.1172378594601703,0.3218560332068502,0.8403014241905424,0.7456026104025554,0.7033504204374504,0.0185333858330424,0.2045204631073961,0.9161547976155554,0.3618931487956518,0.9039562460519173,0.5424886925809498,0.1844982077295418,0.1263222327112790,0.7593345794637383,0.1827551201836928,0.8802202762478126,0.1705239541691373,0.2256800971113518,0.2733819378881632,0.3173157760488409,0.1645718171096276,0.6675153396406747,0.5660143450675598,0.3578814768036276,0.6307952495435231,0.8873576279205073,0.3662894095137199,0.5048084903996477,0.2281901120339474,0.5219606103012155,0.0475490088796006,0.8317696209213552,0.7947877067116966,0.9097557887014727,0.6509597704983129,0.3934602864987498,0.7848193528059960,0.2539072093804866,0.3775039698823839,0.7895124581593614,0.1073406464920103,0.0003694538028769,0.9072659238741106,0.7102228331892859,0.9371693054852862,0.2836931814829322,0.7240295790713418,0.1918404061309250,0.4852350407676935,0.9773878501622880,0.2131125075803663,0.4070326231452788,0.6975427110202344,0.3072969272301052,0.8679227590877204,0.4497225198194954,0.7558416881392902,0.4824199473869148,0.1148073608590324,0.8159927925169433,0.6522439763193224,0.6997119727077484,0.6879646855816079,0.8399325384944362,0.8564072371723165,0.7107190716595946,0.3897105010178454,0.2165664440098062,0.1083748089654254,0.7892948336849434,0.0954009234418166,0.9166167447886511,0.8481761272289677,0.4838694797288019,0.2099751295568306,0.1642023633067507,0.7602494157665640,0.8557915118782744,0.4207121713183412,0.3471020680605910,0.1633280488491655,0.1744490033827950,0.0195734496319542,0.2508022618716593,0.3088481027208493,0.6405163857343217,0.1342269099011211,0.4874907794815911,0.0418330296137524,0.2012372506788175,0.6376185983594594,0.3023994054190812,0.1390998485214542,0.5615111773654405,0.1372684818400389,0.4076286737842619,0.3124047682212688,0.0673333853796745,0.8538155960169694,0.2264502338256920,0.8939826804650864,0.5074631350615357,0.0834655971654996,0.6959402070827505,0.8819869267204717,0.2964957627917154,0.5588564959163109,0.2136732312914326,0.0973217976732746,0.7037203957809697,0.6894731040529316,0.9000501762610165,0.0617077760685737,0.7677052927984415,0.6526647436677779,0.4777949729365273,0.6801385230757941,0.4371624237099487,0.5310844357735870,0.2158908514379947,0.5764921617584733,0.9022197215362544,0.1747334143960538,0.9071375582866079,0.1516762353254837,0.7930015180227352,0.7775168962671968,0.2866649498635274,0.3466009978887631,0.8023464557725686,0.8517975950854821,0.6929160303868892,0.0019759158613048,0.1942619374926490,0.4531193875955042,0.6558649137876298,0.0909834062172954,0.3236332425492039,0.3688153351511879,0.0123523399291338,0.0816598898180108,0.9205536786096888,0.3901689818083165,0.3381126338327830,0.5117641466258859,0.7375684220984432,0.2406916293505075,0.3713945557230128,0.9088464336976625,0.6594735089035115,0.7274901507084675,0.8752423445113200,0.5362489496060877,0.6379247445789745,0.6499580720672189,0.9917629589288325,0.3327297206654817,0.1763280388788917,0.5442639717572666,0.0889854086977362,0.5189788665245189,0.2721915460527836,0.8670722334026697,0.2949753419007060,0.8519228006954875,0.9965570736660422,0.8980742603997116,0.8674458385759246,0.3145859052029372,0.9967998298801479,0.3868115667192319,0.3565052805265903,0.0683470885587610,0.5187320958444533,0.1342309616199839,0.6559384831487848,0.5120507397279379,0.8366207805632708,0.3953734116607221,0.4141078132270407,0.5523098886722280,0.4061223405441839,0.3778185161658650,0.6871274889852514,0.0748563050641009,0.9765552505741619,0.1566670807808018,0.3640511712823301,0.5443038654254303,0.4613564286666719,0.3231351931221482,0.9146553673384035,0.7793692707919372,0.2798299264534516,0.4933734734046150,0.8094683437652272,0.0534814452070191,0.0951936399076104,0.4861898331372952,0.5152070729598438,0.8394941616987318,0.3732457907745828,0.0568086505200754,0.9120466038175147,0.2726619421842796,0.3709848701818776,0.8068952559525592,0.0175168537616343,0.5036937829589907,0.9940195889184341,0.4797122192008942,0.4961089401022107,0.7809546272181692,0.1301561585302261,0.5366755200255083,0.1128565259803350,0.8943730298869187,0.1799447444174182,0.2201190368366050,0.8753675501213257,0.8398899928852404,0.5340230891173814,0.3231419731504945,0.8532294765362655,0.6736646367580000,0.4721561993808282,0.5771360097346530,0.7885171621053093,0.0253586224398383,0.3247626178547565,0.6024570379417656,0.4168570998203274,0.3504309474259760,0.8801663387008785,0.5746136515283089,0.1790640978976450,0.3493136900241086,0.4657719123483505,0.4144655468009716,0.7038714348822231,0.1696599946216028,0.1391502270191676,0.8603573883233392,0.5502842765069959,0.9816442094657777,0.8270262530199375,0.1337007401202343,0.6492131122617112,0.7431544064279435,0.3805169474242801,0.9150953138783089,0.8735367007896008,0.8750746030710050,0.6108222830159694,0.6753170800746033,0.3054710725813505,0.0501038176240883,0.2035791739838101,0.2383819670595145,0.8005057428034518,0.7938488604472249,0.0183780938472497,0.9921582313217957,0.1789311651042342,0.3915625509766686,0.0628551193805668,0.1456779926762348,0.9007882885172908,0.5555425070019170,0.3576114221278630,0.7635428359562264,0.4286011175385681,0.7654791976164463,0.5162476019543817,0.7057075554997231,0.7007397658660730,0.6736657007940422,0.7728576966434986,0.8715852670704878,0.8466383837380625,0.3384554592605937,0.9279228974729415,0.0453627556773660,0.6448416750155583,0.4096673039764480,0.7289203371521643,0.5417824967493221,0.7396086644100067,0.2048492586262753,0.2691425789469585,0.1289602802735569,0.1457345160402984,0.2273899452888360,0.6139598039975203,0.9100225744349989,0.1512819007743532,0.1469919956973716,0.6814262232191052,0.1587217255303272,0.9187890900852107,0.6813482603437024,0.2329124516029434,0.0936706052597941,0.3855429843000802,0.6169741114680534,0.4864941963397404,0.1080575031731545,0.3588270011166237,0.9051147275162463,0.9745773142085307,0.6318053717873084,0.2772461209805898,0.3319939069133224,0.3917435833214519,0.4620502835428577,0.8659259629742828,0.9730153381698836,0.3473165563062378,0.7692638611277863,0.6626422138245039,0.5210726226312445,0.4060693282662281,0.6959390298910156,0.2863999280549585,0.2286511418543063,0.6178083199159283,0.2012111722497321,0.1515193936189262,0.6062250275193830,0.5544256547253700,0.5537477701687009,0.9922394775749370,0.6141359711131713,0.9527961769852771,0.1652901233943599,0.1055430076576504,0.8342522922131475,0.6598197713772859,0.0278675635475048,0.9231731076367075,0.6208628339790101,0.1829554956326985,0.8344939368937605,0.2302719444177448,0.6373372071596501,0.8517141592929672])

In [ ]:
from CSRandom import CSRandomLite

In [ ]:
def f(seed, day, steps):
    r = CSRandomLite(seed + 1 + day*10 + steps)
    for _ in range(day):
        r.Sample()
    dotd = r.Next(194, 240)
    for _ in range(2):
        r.Sample()
#     if dotd == 217:
#         for _ in range(2):
#             r.Sample()
    return r.Next(-100,101) / 1000.

In [ ]:
nseeds = 100000

start_day = 4
start_steps = 3
start_luck = np.zeros(nseeds)
for seed in range(nseeds):
    start_luck[seed] = f(seed, start_day, start_steps)

end_day = 5
end_offset = 4
end_step_addons = np.arange(0,10, dtype=int)
results = np.zeros((nseeds, len(end_step_addons)))
for step_idx in range(len(end_step_addons)):
    end_step_addon = end_step_addons[step_idx]
    for seed in range(nseeds):
        results[seed, step_idx] = f(seed, end_day, start_steps+end_offset+end_step_addon)

In [ ]:
bins = [-0.1, -0.07, -0.02, 0.02, 0.07, 0.1]
for idx in range(len(end_step_addons)):
    hist, _, _ = np.histogram2d(start_luck, results[:,idx], bins=bins)
    plt.imshow(hist)
    plt.xticks(np.arange(5),['Skull','Bat','Neutral','Gold','Star'])
    plt.yticks(np.arange(5),['Skull','Bat','Neutral','Gold','Star'])
    plt.title('Extra steps {}'.format(end_step_addons[idx]))
    plt.xlabel('Day 5 Luck')
    plt.ylabel('Day 6 Luck')
    plt.show()

In [ ]:
hist1, _, _ = np.histogram2d(start_luck, results[:,0], bins=bins)
hist2, _, _ = np.histogram2d(start_luck, results[:,-1], bins=bins)


In [ ]:
hist1-hist2

In [3]:
monster_mushroom_seeds

[5484,
 5485,
 5838,
 5839,
 8248,
 8249,
 15510,
 15511,
 19884,
 19885,
 22648,
 22649,
 29556,
 29557,
 32320,
 32321,
 36694,
 36695,
 36900,
 36901,
 43956,
 43957,
 46366,
 46367,
 50740,
 50741,
 53504,
 53505,
 60412,
 60413,
 60766,
 60767,
 63176,
 63177,
 63324,
 63325,
 70438,
 70439,
 74812,
 74813,
 77016,
 77017,
 77222,
 77223,
 77370,
 77371,
 77576,
 77577,
 79780,
 79781,
 84484,
 84485,
 87042,
 87043,
 87248,
 87249,
 91416,
 91417,
 91622,
 91623,
 94180,
 94181,
 98530,
 98531,
 101088,
 101089,
 101294,
 101295,
 103852,
 103853,
 105668,
 105669,
 108226,
 108227,
 108432,
 108433,
 115340,
 115341,
 117898,
 117899,
 119714,
 119715,
 122272,
 122273,
 122478,
 122479,
 125036,
 125037,
 129386,
 129387,
 131944,
 131945,
 132298,
 132299,
 134708,
 134709,
 139082,
 139083,
 145990,
 145991,
 146344,
 146345,
 148754,
 148755,
 149108,
 149109,
 156016,
 156017,
 158780,
 158781,
 163154,
 163155,
 170062,
 170063,
 172620,
 172621,
 172826,
 172827,
 175384,

# Animal Mechanics

### Pig

```
bool produceToday = 
    (this.daysSinceLastLay < this.daysToLay - (!this.type.Equals("Sheep") || 
    !Game1.getFarmer(this.ownerID).professions.Contains(3) ? 0 : 1) || 
    r.NextDouble() >= (double)this.fullness / 200 ? 
        false : 
        r.NextDouble() < (double)this.happiness / 70);
```